In [ ]:
!pip install transformers datasets evaluate rouge-score peft accelerate bitsandbytes --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
  

In [ ]:
# Cell 2: Load Data
import pandas as pd

train_df = pd.read_csv("train.csv")  # Must contain 'Prompt' and 'Clinician' columns
test_df = pd.read_csv("test.csv")    # Must contain 'Prompt' column

# Verify columns
assert all(col in train_df.columns for col in ['Prompt', 'Clinician'])
assert 'Prompt' in test_df.columns

In [ ]:
train_df.head(5)

,Master_Index,County,Health level,Years of Experience,Prompt,Nursing Competency,Clinical Panel,Clinician,GPT4.0,LLAMA,GEMINI,DDX SNOMED
0,ID_VBWWP,uasin gishu,sub county hospitals and nursing homes,18.0,i am a nurse with 18 years of experience in ge...,pediatric emergency burns,surgery,summary a 4 year old with 5 superficial burns ...,given your vast experience as a nurse in uasin...,1 immediate treatment protocol for second degr...,here s a response addressing the questions reg...,288514009 burn involving 5 percent of body sur...
1,ID_XMBBY,uasin gishu,national referral hospitals,17.0,i am a nurse with 17 years of experience in ge...,child health,paediatrics,summary 6 year old present with vomiting and a...,clinical summary • a 6 year old girl with know...,based on the symptoms and signs you ve describ...,based on the presentation the 6 year old girl ...,420270002 ketoacidosis due to type 1 diabetes ...
2,ID_JZNZW,kiambu,sub county hospitals and nursing homes,12.0,i am a nurse with 12 years of experience in ge...,general emergency,internal medicine,summary a 47 year old man presents with severe...,in this case you re dealing with a 47 year old...,firstly i must commend you on your thorough hi...,this 47 year old male presenting with severe r...,13200003 peptic ulcer disorder 25458004 acute ...
3,ID_QOQTK,uasin gishu,national referral hospitals,12.0,i am a nurse with 12 years of experience in pr...,critical care,internal medicine,summary 72 year old female with inability to w...,given er s clinical presentation and vitals th...,to me with this query based on the information...,this 92 year old female patient er presents wi...,14760008 constipation finding 419284004 altere...
4,ID_ZFJBM,uasin gishu,national referral hospitals,16.0,i am a nurse with 16 years of experience in ge...,adult health,internal medicine,a 22 year old female presents with headache di...,the 22 year old female patient is presenting w...,thank you for presenting this case based on th...,this 22 year old female patient presents with ...,95874006 carbon monoxide poisoning from fire d...


In [ ]:
test_df.head(5)

,Master_Index,County,Health level,Years of Experience,Prompt,Nursing Competency,Clinical Panel
0,ID_CUAOY,uasin gishu,sub county hospitals and nursing homes,2.0,i am a nurse with 2 years of experience in gen...,adult health,surgery ent
1,ID_OGSAY,kiambu,sub county hospitals and nursing homes,22.0,i am a nurse with 22 years of experience in ge...,child health,surgery
2,ID_TYHSA,uasin gishu,national referral hospitals,NaN,i am a nurse working in a national referral ho...,general emergency,internal medicine
3,ID_CZXLD,kakamega,dispensaries and private clinics,NaN,i am a nurse working in a dispensaries and pri...,child health,paediatrics
4,ID_ZJQUQ,kakamega,health centres,NaN,i am a nurse working in a health centres in ka...,child health,paediatrics


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Master_Index         337 non-null    object 
 1   County               337 non-null    object 
 2   Health level         337 non-null    object 
 3   Years of Experience  252 non-null    float64
 4   Prompt               337 non-null    object 
 5   Nursing Competency   337 non-null    object 
 6   Clinical Panel       337 non-null    object 
 7   Clinician            337 non-null    object 
 8   GPT4.0               336 non-null    object 
 9   LLAMA                336 non-null    object 
 10  GEMINI               336 non-null    object 
 11  DDX SNOMED           335 non-null    object 
dtypes: float64(1), object(11)
memory usage: 31.7+ KB


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Master_Index         100 non-null    object 
 1   County               100 non-null    object 
 2   Health level         100 non-null    object 
 3   Years of Experience  75 non-null     float64
 4   Prompt               100 non-null    object 
 5   Nursing Competency   100 non-null    object 
 6   Clinical Panel       100 non-null    object 
dtypes: float64(1), object(6)
memory usage: 5.6+ KB


In [ ]:
# Cell 3: Convert to Hugging Face Dataset
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# Cell 3: Basic Tokenization
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Simple tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["Prompt"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )

# Convert to Hugging Face Dataset
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df).map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test_df).map(tokenize_function, batched=True)

# Add labels
def add_labels(examples):
    examples["labels"] = tokenizer(
        examples["Clinician"],
        max_length=256,
        truncation=True,
        padding="max_length"
    )["input_ids"]
    return examples

train_dataset = train_dataset.map(add_labels, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/337 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/337 [00:00<?, ? examples/s]

In [ ]:
# Cell 4: Load Base Model
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")  # Small model for quick start

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Cell 5: Training Setup
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_strategy="no",
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [ ]:
# Cell 6: Train (Optional)
trainer.train()  # Skip this if you just want to test inference

Step,Training Loss


TrainOutput(global_step=255, training_loss=6.69254270067402, metrics={'train_runtime': 222.521, 'train_samples_per_second': 4.543, 'train_steps_per_second': 1.146, 'total_flos': 692289683324928.0, 'train_loss': 6.69254270067402, 'epoch': 3.0})

In [ ]:
# Cell 7: Generate Predictions
def predict(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").input_ids
    inputs = inputs.to(model.device)
    outputs = model.generate(inputs, max_length=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test prediction
sample_pred = predict(train_df["Prompt"].iloc[0])
print(f"Prompt: {train_df['Prompt'].iloc[0][:100]}...")
print(f"Predicted: {sample_pred}")
print(f"Actual: {train_df['Clinician'].iloc[0][:100]}...")

Prompt: i am a nurse with 18 years of experience in general nursing working in a sub county hospitals and nu...
Predicted: summary a 4 year old child check with the to a hot stove the child summary to the to to the to a dominating the child summary the to to the to the to to the to the to the to to the to the to the to to the to to the to to the to the to to to the to the to the to to the to to the to to the to the to to the to to the to to the to the to to the to the to the to to the to to to the to the to to the to to the to to the to to the to to to the to to the to to to to the to the to to to the to to to to the to the to to the to the to to to the to to the to to the to to the to to to the to the to the to the to to to the to to the to to to to the to the to to the to to the to to the to to to the to to the to to the to the to to to the to to the to to to the to to to the to the to to the to to to the to to to the to the to to to the to to the to to the to to the to to to the
Act

In [ ]:
# Cell 8: Create Submission
test_df["Clinician"] = test_df["Prompt"].apply(predict)
test_df[["Master_Index", "Clinician"]].to_csv("submission.csv", index=False)

from google.colab import files
files.download("submission.csv")

KeyboardInterrupt: 